# K Nearest Neighbours

In [9]:
# Function for distance between attribute vectors and mode
dist <- function(v1, v2){
  return(sqrt(sum((v1-v2)^2)))
}
mode <- function(lab,dist){
  return(names(which.max(table(lab))))
}

In [10]:
# function to calculate overall, positive, negative, accuracy and MCC
CalcAccus <- function(pred, true, k){
    tab = table(true, pred)
    TP = tab[1,1]
    FP = tab[2,1]
    FN = tab[1,2]
    TN = tab[2,2]
    
    accu_o = (TP+TN)/(TP+TN+FP+FN)
    accu_P = TP/(TP+FN)
    accu_N = TN/(TN+FP)
    MCC    = (TP*TN - FP*FN)/sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    
    x <- data.frame("k" = k, "overall" = accu_o, "positive" = accu_P, 
                    "negative" = accu_N, "MCC" = MCC)     
    return(x)
}

### Function for KNN
#### Input
- data:  input data in form of data frame with labels in last col  
- obj:   input objects {set of attributes} for which predictions are to be made; also a data frame with one or more rows with attributes  
- k:     algorithm parameter 

#### Output
- List to predicted labels

In [11]:
knn <- function(data, obj, k){
    x_data = data[ ,-length(data)]
    y_data = data[ ,length(data)]
    label = c()
    for(each in 1:nrow(obj)){
        distance = c()
        for(row in 1:nrow(data)){
            distance[row] = dist(x_data[row,], obj[each,])
        }
        sort_order = order(distance)
        sorted_label = y_data[sort_order][1:k]
        sorted_distance = distance[sort_order][1:k]
        label[each] = mode(sorted_label,sorted_distance)        
    }
    return(label)
}

In [12]:
# data pre-prosessing - reading data, removing unnecessary cols, moving labels to last col
data <- read.csv("data.csv", na.strings = c('',"NA"))
data = data[,c(-1,-ncol(data))]
data = data[,c(2:ncol(data),1)]

In [13]:
head(data,5)

radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave.points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave.points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,M
20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,M
19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,M
11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,M
20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,M


In [14]:
# shuffling the data points and spliting in test and train data at random
shuf  = sample(2, nrow(data), replace = T, prob = c(0.8,0.2))
train = data[which(shuf == 1), ]
test  = data[which(shuf != 1), ]

x_test = test[ ,-length(train)]
y_test = test[ ,length(train)]

In [15]:
# loop to findout the performance matrix for different K values
k_set = seq(1,13,2)
k_accus = data.frame()
for(k in k_set){
    y_pred = knn(train, x_test, k)
    accus  = CalcAccus(y_pred, y_test, k)
    k_accus = rbind.data.frame(k_accus, accus)
}

In [16]:
k_accus

k,overall,positive,negative,MCC
1,0.9122807,0.9305556,0.8809524,0.8115079
3,0.9298246,0.9583333,0.8809524,0.8483106
5,0.9298246,0.9583333,0.8809524,0.8483106
7,0.9210526,0.9444444,0.8809524,0.8296603
9,0.9210526,0.9444444,0.8809524,0.8296603
11,0.9298246,0.9583333,0.8809524,0.8483106
13,0.9298246,0.9583333,0.8809524,0.8483106
